# Übungen zu Kapitel 11.5 - Einsatz von KI für die Fehlervorhersage
**(Aufbau eines Fehlervorhersagesystems)**

*Eine Übung zum Buch "[Basiswissen KI-Testen - Qualität von und mit KI-basierten Systemen](https://dpunkt.de/produkt/basiswissen-ki-testen/)", ISBN 978-3-86490-947-4*

In dieser Übung kannst du ein Fehlervorhersagesystem trainieren, das anhand von Codemetriken prognostizieren soll, ob ein Programmcode vermutlich fehlerhaft ist oder korrekt arbeitet. Dabei wirst du viele Arbeitsschritte, die du in den vorausgegangenen Übungen kennengelernt hast, erneut anwenden: _Datenvorbereitung_, _Modellauswahl_, _Training_, _Evaluierung_, _Tuning_ und _Test_.

Diese Übung besteht aus sieben Aufgaben, die aufeinander aufbauen
* Aufgabe 1: Das Zielverstehen
* Aufgabe 2: Algorithmus wählen
* Aufgabe 3: Datenvorbereitung
* Aufgabe 4: Aufteilen der Daten und Modellgenerierung
* Aufgabe 5: Mehr Aufbereitung der Daten
* Aufgabe 6: Modelle mit den zusätzlich aufbereiteten Daten generieren
* Aufgabe 7: Unausgeglichene Daten angleichen

Wir nutzen hier wieder verschiedene Bibliotheken mit passenden Methoden, die du bereits kennengelernt hast:

[<img src="https://pandas.pydata.org/docs/_static/pandas.svg" alt="pandas" width="80" height="24">](https://pandas.pydata.org/docs/reference/index.html)
&emsp; [<img src="https://seaborn.pydata.org/_static/logo-wide-lightbg.svg" alt="seaborn" width="100" height="24">](https://seaborn.pydata.org/)
&emsp; [<img src="https://docs.scipy.org/doc/scipy/_static/logo.svg" alt="SciPy" width="24" height="24"> SciPy](https://docs.scipy.org/doc/scipy/index.html)
&emsp; [<img src="https://scikit-learn.org/stable/_static/scikit-learn-logo-small.png" alt="Scikit-learn" width="80" height="24">](https://scikit-learn.org/stable/modules/classes.html)
&emsp; [<img src="https://matplotlib.org/_static/logo_light.svg" alt="Matplotlib" width="100" height="24">](https://matplotlib.org/)
<!-- &emsp; [<img src="https://numpy.org/doc/stable/_static/numpylogo.svg" alt="Numpy" width="80" height="24">](https://numpy.org/doc/stable/reference/index.html#reference) -->

*Hinweis 1:* Wenn du das Prinzip unseres Beispiels verstanden hast, kannst du dir bei Interesse über das GitHub Projekt [NASADefectDataset](https://github.com/klainfo/NASADefectDataset) weitere Datensätze ansehen und diese als zusätzliche Quelle heranziehen...

*Hinweis 2:* In den folgenden Codebeispielen werden sehr häufig Pseudo-Zufallszahlengeneratoren eingesetzt. Damit unsere Hinweise und Kommentierungen zum Code und dessen Ergebnissen passen, belegen wir die Startwerte dieser Generatoren mit festen Werten. In echten Projekten geht man so nicht vor. Für eine möglichst hohe Reproduzierbarkeit in der Entwicklungskette, solltest du dann aber zumindest die zufälligen Startwerte protokollieren.

## Aufgabe 1: Das Ziel verstehen
Du sollst ein Fehlerprognosesystem für Softwaremodule entwickeln. Dieses soll anhand von Codemetriken, die zu einem Softwaremodul ermittelt werden, abschätzen, ob das Softwaremodul noch Fehlerzustände enthalten könnte oder nicht. Als Grundlage dienen dir Datensätze von Softwaremodulen mit und ohne Fehlerzustände, für die eine Reihe von Codemetriken bereits ermittelt wurden.

* Welche Fragen solltest du im Vorfeld klären?
* Wie aussichtsreich vermutest du ist es, aus Codemetriken fehlerhaften Programmcode zu prognostizieren?

Eine Diskussion möglicher Antworten findest du wieder hinter der folgenden Zelle (...)

**Lösungsdiskussion:** Wichtige Fragen, die vorab mit Nutzern, Kunden, Geldgebern oder anderen Verantwortlichen oder Betroffenen geklärt werden sollten, sind zum Beispiel:
* *Wie und für wen soll das System eingesetzt werden?*</br>
Ist es eine Art Hinweisgeber für Entwickler, der vor jedem Code-Checkin erfolgt? Sollen tausende, bereits existierende Module gescannt werden? Die Vor- und Nachbereitungen können sich unterscheiden, ebenso wie eine spätere Einbettung in ein größeres Gesamtsystem. Je nach Nutzerkreis können unterschiedliche Erwartungen an die funktionale Leistung des Systems existieren.
* *Auf welchen Codequellen soll das System eingesetzt werden?*</br>
Welche Programmiersprachen soll das System analysieren können? Um welche Produkte und Domänen handelt es sich bei den Software-Quellen? Codemetriken können je nach Programmiersprache unterschiedliche Kritikalität für noch enthaltene Fehlerzustände signalisieren. Je nach Anwendungsgebiet können hohe oder eher niedrige Erwartungen an die funktionale Leistung gestellt werden.
* *Welche funktionalen Leistungsmetriken sind besonders wichtig?*</br>
Nicht jede Domäne und jeder Stakeholder haben die gleichen Bedürfnisse. Manchmal ist die Genauigkeit des Systems gefragt. Häufiger könnte jedoch die Sensitivität oder Präzision von größerer Bedeutung sein. Dies vorher zu klären, kann falsch spendierten Aufwand im Tuning reduzieren.

Programmcode mit Fehlerzuständen allein aus den Codemetriken zu prognostizieren, könnte ein schwieriges Unterfangen sein. Beispielsweise muss eine besonders hohe [zyklomatische Komplexität](https://de.wikipedia.org/wiki/McCabe-Metrik) (nach McCabe) nicht gleichbedeutend mit einem Fehlerzustand sein. Die zyklomatische Komplexität könnte nach einer Fehlerkorrektur gleichgeblieben sein, weil sie nicht ursächlich für den Fehler war. Eine statistische Häufung von Fehlern bei komplexen Modulen könnten wir uns aber vorstellen.

Entscheidend ist sicher auch, ob unsere Trainingsdaten mit Beispielen aus geeigneten Entwicklungsständen erhoben wurden: erste Versionen oder "abgehangene" Software, Code von unerfahrenen oder erfahrenen Teammitgliedern.

## Aufgabe 2: Algorithmus auswählen


Nimm an, aus Aufgabe 1 haben wir folgendes Ziel vereinbart: Mit mehreren bereits vorhandenen Listen von Softwaremodulen - mit und ohne bekannte Fehlerzustände - und ihren dazu ermittelten Codemetriken sollen wir ein Modell trainieren, das uns diejenigen Module identifizieren soll, in denen noch Fehler vermutet werden. Anhand dieser Vermutung sollen die Module nochmal einem intensiven Codereview und weiteren Tests unterzogen werden.

**Aufgabe:** *Welche Art von maschinellem Lernen (ML) würdest du wählen?</br>*
Erinnere dich an die erst Aufgabe aus [Übung zu Kapitel 3.3 - Wahl der passenden ML-Art](../Kap03.3_ML-Art_wählen/Übung_ML-Arten.ipynb).

**Lösung:** Die Beschreibung deutet klar auf eine **Klassifikation**. Denn wir haben Trainingsdaten zur Verfügung, zu denen die erwarteten Ergebnisse bekannt sind (mit oder ohne Fehlerzustand). Die Daten sind also bereits gekennzeichnet (annotiert/gelabelt). Als Ergebnis wird eine einfache binäre Entscheidung, ob ein Fehlerzustand enthalten ist, erwartet: *ja* oder *nein*. Also entwickeln wir einen Klassifikator.

## Aufgabe 3: Datenvorbereitung

### Schritt 1: Datenbeschaffung
*siehe auch Kapitel 4.1.1 im Buch*

Im Verzeichnis `Data/` dieser Übung findest du **drei Dateien** im _arff_-Dateiformat. Zum Laden, Bearbeiten und Visualisieren dieser Daten laden wir zunächst die Bibliotheken: *pandas*, *seaborn*, *matplotlib* und *scipy* (letztere, damit wir das *arff*-Datenformat lesen können). Einige brauchen wir aber erst später.

In [ ]:
import pandas            as pd
import seaborn           as sns
import matplotlib.pyplot as plt
from   scipy.io      import arff

Mit der oben von *scipy* importierten Klasse [arff](https://docs.scipy.org/doc/scipy/reference/io.html#module-scipy.io.arff) kannst du mit der Methode [loadarff](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.arff.loadarff.html#scipy.io.arff.loadarff) jeweils eine der Dateien laden. Sieh dir in der verlinkten Dokumentation zu der Methode an, welche Daten diese zurückgibt. Was wir tun wollen, ist:
* alle drei *'NASA bug'*-Dateien in drei separate Datenobjekte `data1` ... `data3` laden und
* diese Objekte in [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas.DataFrame)s der *pandas*-Library (`df1` ... `df3`) konvertieren. Pandas DataFrame-Objekte sind sehr vielseitige Klassen, weshalb wir unsere Daten in genau dieser Struktur vorhalten und bearbeiten wollen.
* Prüfen, ob wir die Daten richtig geladen haben.

Ersetze die `...` im Code wieder durch die richtigen Einträge.

In [ ]:
data1, meta1 = arff.loadarff(...) # NASA bug data 1 laden
df1 = pd.DataFrame(data1)         # und die Daten in einen DataFrame konvertieren

... # ebenso für bug data 2 und 3.

print(df1.shape) # gibt die Dimension des Datensatzes aus, um die Zahl der Instanzen (Einträge, Zeilen) und der Attribute anzuzeigen.
print(...)
print(...)

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung01.py

Wir haben also drei Dateien mit 3619, 3630 und 2774 Einträgen (Zeilen) und 21 Werten pro Eintrag (Attribute/Ergebnisse).

Wir wollen aber alle drei Datensätze zusammen betrachten und diese daher in einem einzigen Datensatz `df` vereinigen.

Passen die Datensätze überhaupt zusammen? Wenn ja, hänge mit dem folgenden Code, der die Methode [concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)(...) benutzt, die drei Datensätze hintereinander. die Angabe `axis=0` gibt an, dass das Verketten entlang der Achse "0" erfolgt (das ist die Index-Achse entlang der Einträge bzw. Instanzen).</br>
Außerdem haben wir durch die direkt angewendete zweite Methode [reset_index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html)(...) die Nummerierung der Einträge von Null beginnend erneuert. Das `drop=True` im Argument ist notwendig, damit nicht eine zusätzliche "index"-Spalte als Dateneintrag erzeugt wird.

Fülle wieder den Code-Teil `...` passend aus (nutze den obigen Link zur Dokumentation) und schau dir die Ausgabe, die das `df` in der zweiten Zeile erzeugt, an. *Stimmt alles?*

In [ ]:
df = pd.concat(..., axis=0).reset_index(drop=True)
df

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung02.py

Dir sollte aufgefallen sein, dass in der Ausgabe oben "10023 rows x **23** columns" steht, obwohl doch unsere drei ursprünglichen Datensätze nur **21** Spalten besitzen - das hat uns die Ausgabe durch `print(df1.shape)` wiedergegeben.

Grund ist, dass hier *zwei neue* Spalten dazu gekommen sind: **T** und **defect**. Beide Spaltennamen kommen *nur* in `df3` vor aber *nicht* in `df1`, so dass die *concat()*-Methode annimmt, wir hätten andere Attribute als zuvor und neue Spalten anlegt.

Schaue dir die Spaltennamen von `df1` und `df3` an:

In [ ]:
df1.columns, df3.columns

Hier liegt anscheinend unterschiedliche Schreibweisen vor. Der letzten Datensatz hat statt "t" ein "T" als Spaltenname (=Name des Attributs bzw. der Codemetrik) und "defect" statt "defects" als Spaltenname für die Zielgröße verwendet.

Wir müssen also die Namen der Spalten *vor* dem Zusammenführen korrigieren. Dies tun wir für den letzten Datensatz `df3` mit der Methode [rename](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)(...). Dem Argument `columns=` übergeben wir ein Python-Dictionary - `{alt:neu, ... }` - das den alten auf den neuen korrigierten Namen enthält. Dar Argument `inplace` bewirkt, dass die Korrektur direkt im Datensatz `df3` erfolgt.

Fülle den `...`-Teil mit dem Namens-Mapping aus.

In [ ]:
df3.rename(columns = ..., inplace=True) # korrigiere abweichende Spaltennamen

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung03.py

Nun können wir nochmal die Verkettung der Datensätze durchführen:

In [ ]:
df = pd.concat([df1, df2, df3], axis=0).reset_index(drop=True)

Machen wir einen Test, ob die Verkettung - zumindest von den Dimensionen des Datensatzes - funktioniert hat. Der folgende Code sollte `True` ergeben:

In [ ]:
df.shape == (10023, 21)

### Schritt 2: Datenvorverarbeitung
*siehe auch Kapitel 4.1.2 im Buch*

Bei der Datenvorverarbeitung kümmern wir uns um die Inhalte der Daten, indem wir sie bereinigen, umwandeln, anreichern oder auch Stichproben daraus erheben.

#### Fehlende Werte finden und entfernen

Bei der Bereinigung suchen wir zum Beispiel nach *falschen* oder *fehlenden* Werten, korrigieren diese, ersetzen sie oder verwerfen sie als fehlerhaft.

In unserem Datensatz `df` suchen wir nach Einträgen mit *NaN*-Werten (also *Not-a-Number*). Das DataFrame-Objekt kennt hier die Methode [isna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html#pandas.DataFrame.isna)().
Diese gibt einen DataFrame der gleichen Größe - jedoch nur mit *True*- oder *False*-Werten - zurück. *True* für *NaN*-Werte im ursprünglichen DataFrame.

Da der Datensatz zu groß ist (über 200.000 Werte), um einen visuellen Überblick zu bekommen, benutzen wir zusätzlich die Methode [sum](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html#pandas.DataFrame.sum)(). Diese zählt alle *True*-Werte in jeder Spalte. Um genau zu sein, interpretiert die Methode *False* als 0 und *True* als 1 und summiert diese Zahlen pro Spalte auf.

In [ ]:
df.isna().sum(axis=0)

Du kannst aus der Ausgabe erkennen, dass die Spalten (Attribute `uniq_Op` bis `branchCount`) jeweils **zwei** Einträge mit *NaN*-Werten haben.

Wir wollen uns nun diese Einträge selbst genauer ansehen. Wir geben durch die folgende Zeile nur diejenigen Einträge in `df` zurück, die mindestens einen *NaN*-Wert enthalten:

In [ ]:
df[df.isna().sum(axis=1)>0]

Wer den obigen Code genauer verstehen möchte, kann die folgenden Zellen (...) durch Anklicken aufdecken.

Wir identifizieren wieder die fehlenden (*NaN*) Werte. Wir bilden die Summen über die Zeilen (axis=1). Diese Summen vergleichen wir mit Null (`>0`) und erhalten so eine Liste, die ein *True* enthält, wenn mindestens ein Wert der Zeile ein *NaN* ist. Benutzen wir diese Liste als Index in den DataFrame, bekommen wir nur diejenigen Zeilen des Datensatzes zurück, deren Index *True* ist.

In [ ]:
values_with_na = df.isna()                   # Gibt einen DataFrame - so groß wie `df` - zurück, der nur an den Stellen ein "True" enthält,
                                             # die in `df` einen "NaN"-Wert haben. Alle anderen Einträge sind "False".
num_of_na      = values_with_na.sum(axis=1)  # sum() zählt für jeden Eintrag alle "True"-Werte entlang der Achse "1" - also über alle Spalten - und gibt deren Anzahl für jeden Eintrag zurück
entry_has_na   = num_of_na>0                 # Der Vergleich ">0" gibt ein "True" für jeden Eintrag zurück, der größer als 0 ist (also in df mindestens einen "NaN"-Wert enthält)
df[entry_has_na]                             # gibt diejenigen Einträge aus `df` zurück, die in `entry_has_na` ein "True" enthalten.

Um diese beiden Einträge aus dem Datensatz zu entfernen, bedienen wir uns der vorgefertigten Methode [dropna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html#pandas.DataFrame.dropna)(). Diese entfernt alle Einträge bei denen mindestens ein *NaN*-Wert - durch das Argument `how='any'` festgelegt - vorliegt:

In [ ]:
df.dropna(axis=0, how='any', inplace=True)
df[2142:2246]

Aus dem ausgegebenen Teil des Datensatzes erkennst du, dass die beiden Einträge mit den Indizes **2144** und **2245** entfernt wurden.

#### Auf Duplikate prüfen und diese entfernen

Ebenfalls Teil der Bereinigung ist, Duplikate aus den Datensätzen zu entfernen. Der *pandas*-DataFrame besitzt auch hierfür eine zugeschnittene Methode: [drop_suplicates](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html#pandas.DataFrame.drop_duplicates)().

Bevor wir diese anwenden, interessiert uns aber wie viele Duplikate es aber überhaupt im Datensatz gibt. Diese gibt uns die Methode [duplicated](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html#pandas.DataFrame.duplicated)() als Index-Vektor zurück. [value_counts]()() zählt - wie vorher bei den *NaN*-Werten - die nur einmal vorkommenden Einträge (*False*) und die Duplikate (*True*):

In [ ]:
df.duplicated().value_counts()    # df.duplicated() gibt einen Vektor zurück, der 'True' für alle erneut vorkommenden _exakten_ Werte-Kombinationen anzeigt.

In [ ]:
df.drop_duplicates(inplace=True)

Der resultierende Datensatz sollte nun nur noch 8927 einmalig vorkommende Einträge enthalten, was wir gleich testen (das Ergebnis sollte `True` sein):

In [ ]:
df.shape == (8927, 21)

#### Transformation

Für einige Algorithmen ist es vorteilhaft mit numerischen Eingabe- und Ausgabewerten zu arbeiten. Wir haben als *defects*-Spalte **kategorische** Werte, die als Zeichenketten vorliegen und die wir in **Zahlen** transformieren. Ein `'false'` soll zu einer `0` werden und ein `'true'` zu einer `1`. Da wir nur in der letzten Spalte Zeichenketten haben, können wir gefahrlos die Methode [replace](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html#pandas.DataFrame.replace)() für unseren Datensatz benutzen. Die ersetzt *alle* Werte, die auf das Argument `to_replace=` passen.

Gib für das Argument `to_replace=` ein Dictionary an, das - ähnlich zur oben verwendeten Methode rename() - alte und neue Werte angibt. Ersetze die Werte direkt im Datensatz (inplace).

In [ ]:
df.replace(to_replace=...,  inplace=...)

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung04.py

Sehen wir uns das Ergebnis an, sollte die letzte Spalte **defects** nur noch `0` und `1` Werte enthalten.

In [ ]:
df

### Schritt 3: Merkmalsermittlung
*siehe auch Kapitel 4.1.3 im Buch.*

#### Eingabe- und Ausgabewerte aufteilen

Zunächst teilen wir unseren Datensatz in **Eingabe-** (X) und **Ausgabe**werte (y) auf. **X** erstellen wir durch das Entfernen der Spalte *defects*, das wir mit der Methode [drop](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html#pandas.DataFrame.drop)() durchführen, und in **y** speichern wir direkt als Kopie die letzte Spalte:

In [ ]:
X = df.drop(columns=...)  # Speichert einen DataFrame, aus dem die Spalte (...) mit den Ausgabewerten entfernt wurde in X
y = df[...]               # Speichert die Spalte (...) mit den Ausgabewereten in y

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung05.py

Die resultierenden Daten prüfen wir wieder auf ihre Größe: Hier sollte `(True, True)` zurückgegeben werden.

In [ ]:
X.shape == (8927, 20), y.shape == (8927,)

#### Merkmale auswählen

Die in unserem Datensatz enthaltenen Attributen sind möglicherweise nicht alle von gleich großer Bedeutung, wenn es darum geht, eine Fehlervorhersage für ein Softwaremodul zu treffen. Doch welche der Attribute eignen sich als Merkmale für das Training, und welche können (oder sollten) entfernt werden?

Wichtig ist, insbesondere diejenigen *Merkmale zu entfernen, die irrelevant für die Problemstellung sind* (z.B. eine Spalte mit einer *laufenden Nummer*, oder der *Dateiname* eines Softwaremoduls). Da wir glücklicherweise keine solchen Spalten in unserem Datensatz haben, brauchen wir uns darum nicht zu kümmern.

Doch wie relevant sind dann die verbleibenden Merkmale?

Mit einer **Korrelation** kann man zwischen zwei Attributen oder Eingabe-/Ausgabewerten numerisch ermitteln, ob und wie stark deren Wertänderungen miteinander in Verbindung (also in Korrelation) stehen.

Wir sehen uns dies zunächst **zwischen allen Eingabewerten** an und *korrelieren* mit der Methode [corr](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html#pandas.DataFrame.corr)() alle 20 Eingabewerte untereinander, was eine Matrix de Dimension 20x20 ergibt. Diese stellen wir mit der *seaborn*-Funktion [heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html)() als farbige Grafik dar.

In der Grafik sehen wir, dass auf der Diagonalen der Matrix die Korrelationswerte alle bei `1` liegen. Das sollte uns nicht verwundern, denn genau hier werden die Eingabewerte jeweils mit sich selbst korreliert!

In [ ]:
plt.figure(figsize=(12,8), dpi=80)    # Wir geben die Größe der Grafik vor
sns.heatmap(X.corr(), annot=True, cmap=plt.cm.Reds);

Wir sehen außerdem den Eingabewert "l" herausstechen, denn hier sind alle Korrelationen mit den jeweils anderen Werten *negativ*. Dies bedeutet aber nur, dass wenn "l" größer wird, häufig die anderen Werte kleiner werden. Wir haben also eine *negative Korrelation*.

Wir interessieren uns jedoch für die **Korrelation** zwischen **Eingabe-** und **Ausgabe**werten, wofür du die Methode [corrwith](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corrwith.html#pandas.DataFrame.corrwith)(...) verwenden sollst.

**Aufgabe:** berechne die Korrelation zwischen X und y, so dass du eine Tabelle mit 20 Eingabegrößen und ihren jeweiligen Korrelationswerten erhältst. Extrahiere aus dieser die *sieben größten* Einträge.</br>
*Hinweis:* Benutze hierbei auch die Methoden [abs](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.abs.html#pandas.DataFrame.abs)(), [sort_values](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html#pandas.DataFrame.sort_values)(
...) und verwende [iloc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html#pandas.DataFrame.iloc)[...], um die 7 größten Ergebniszeilen auszuwählen.

In [ ]:
table = X.corrwith(...)...

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung06.py

Von dieser Tabelle verwenden wir nur die linke *Index*-Spalte (mit den Namen der Attribute), welche wir durch das DataFrame-Attribut [.index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.index.html#pandas.DataFrame.index) erhalten.</br>
Die Index-Liste benutzen wir dann, um die entsprechenden Spalten (Merkmale) aus den Eingabewerten **X** zu extrahieren. Dazu nutzen wir die Methode [filter](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.filter.html)().

Ersetze wieder die `...` durch die Index-Liste der vorhin erzeugten Tabelle `table`:

In [ ]:
Xf = X.filter(...)
Xf

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung07.py

**Hilfestellung:** Hast du bis hierher Probleme gehabt, die Tabelle `table` zu erzeugen, deren Index als Liste darzustellen, oder den Filter auf den Eingabedatensatz X anzuwenden? In der folgenden Zelle (...) findest du eine schnelle Lösung, mit der du trotzdem die nun folgende Aufgabe 4 vollständig bearbeiten kannst.

In [ ]:
Xf = X.filter(items=['uniq_Opnd', 'loc', 'i', 'n', 'total_Opnd', 'total_Op', 'lOCode'])
Xf

## Aufgabe 4: Aufteilen der Daten und Modellgenerierung

### Einen Entscheidungsbaum trainieren (Test-Split)

Als nächstes führen wir die folgenden Schritte aus:
1. Aufteilen der Daten in Trainings- und Validierungsdaten (Die Nutzung der reduzierten Merkmale in `Xf` und die Abtrennung von Testdaten lassen wir zur Vereinfachung außen vor.)
2. Einstellen der Modell-Hyperparameter
3. Trainieren des Modells mit den Trainingsdaten
4. Evaluieren des Modells mit den Validierungsdaten

**Aufgabe:** Sieh dir das Ergebnis an und überlege dir, ob dein trainiertes Modell gut genug ist!
* Welches Problem könnte es geben?
* Woran liegt es?

In [ ]:
from sklearn.model_selection import train_test_split   # zum Aufteilen der Trainings- und Testdaten
from sklearn                 import tree

# 1. Trainings- und Testdaten aufteilen - hier verwenden wir noch ALLE Merkmale!
X_train, X_val, y_train, y_val = train_test_split( X, y, test_size=0.1, random_state=4 )

# 2. Einen Entscheidungsbaum mit maximaler Tiefe ... erzeugen
# 3. und das Modell per '.fit()'-Methode mit den Trainigsdaten trainieren
model_dtree = tree.DecisionTreeClassifier(max_depth=15, random_state=16)
model_dtree.fit(X_train, y_train)

# 4. Die Genauigkeit über alle Ergebnisklassen ermitteln: "Wieviel Prozent aller Ergebnisse waren richtig?"
print('Tiefe des Entscheidungsbaums: {0}'.format(model_dtree.get_depth()))
print('Die Genauigkeit auf den Trainingsdaten    liegt bei {0:3.2f}%'.format(model_dtree.score(X_train, y_train)*100))
print('Die Genauigkeit auf den Validierungsdaten liegt bei {0:3.2f}%'.format(model_dtree.score(X_val  , y_val  )*100))

Zur Erklärung/Lösung die folgende Zelle (...) anklicken:

**Lösung:** Die Genauigkeit liegt mit fast 92% nach dem Training sehr hoch und suggeriert ein gutes Modell.

Aber die Genauigkeit mit den Validierungsdaten ist mit 71.4% deutlich geringer. Hier haben wir es vermutlich mit einem *Overfit* zu tun. Sehen wir uns den 2. Schritt (im Code oben) an. Siehst du, dass die maximale Tiefe des Entscheidungsbaums auf 15 eingestellt wurde. Unser Modell scheint also *"zu groß/komplex"* für die Aufgabe gewählt.

**Folgeaufgabe:** Reduziere *im obigen Code* sukzessive die maximal erlaubte Tiefe des Modells, bis Trainings- und Validierungsgenauigkeit ähnlich groß sind. Ist jetzt unser Modell gut?

Für eine Erklärung/Lösung die folgenden Zellen (...) anklicken:

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung08.py

**Lösung:** Die mit über 77.60% scheinbar immer noch große Genauigkeit täuscht: Wir prüfen zwar mit der Genauigkeit den Anteil aller richtigen Vorhersagen. Bei stark unausgewogenen Anteilen der Ergebnisklassen (*true* und *false*) ist die Genauigkeit aber keine geeignete Metrik - siehe auch [Übung Kapitel 5.4: Evaluierung](../Kap05.4_Evaluierung/%C3%9Cbung_Evaluierung.ipynb)

Warum ist das so?

Sehen wir uns die Statistik des gesamten Datensatzes an:

In [ ]:
y.value_counts()

Wir sehen, dass 6931 Einträge einen erwarteten Ausgabewert `0` (*false*) haben, aber nur 1996 einen Ausgabewert `1` (*true*). Das bedeutet, dass etwa 77.6% (6931/(6931+1996)) aller Ausgabewerte `0` sind.

Ein "Klassifikator", der ungeachtet der Eingabewerte konstant `0` als Ausgabe liefert, würde in allen 6931 "0"-Fällen richtig liegen, und in allen anderen falsch und hätte damit eine Genauigkeit von 77.6%!

Einen solchen Klassifikator kennt auch die *scikit-learn* Bibliothek, den `DummyClassifier`:

In [ ]:
from sklearn import dummy
# 1. Trainings- und Testdaten sind schon aufgeteilt (s.o.)
# 2. Einen DummyClassifier erzeugen, der konstant die häufigste in y_train vorkommende Klasse zurückgibt
# 3. und das Modell per '.fit()'-Methode mit den Trainigsdaten "trainieren"
model_dummy = dummy.DummyClassifier(strategy='most_frequent')
model_dummy.fit(X_train, y_train)

# 4. Die Genauigkeit über alle Ergebnisklassen ermitteln: "Wieviel Prozent aller Ergebnisse waren richtig?"
print('Die Genauigkeit auf den Trainingsdaten    liegt bei {0:3.2f}%'.format(model_dummy.score(X_train, y_train)*100))
print('Die Genauigkeit auf den Validierungsdaten liegt bei {0:3.2f}%'.format(model_dummy.score(X_val  , y_val  )*100))

### Einen Entscheidungsbaum trainieren (10-fach Kreuzvalidierung)

Wir entscheiden uns nun dafür, nicht nur die Genauigkeit, sondern auch die **Präzision** und die **Sensitivität** zu evaluieren. Diese ermitteln wir zudem mit der ***k*-fachen Kreuzvalidierung** (siehe auch [Übung Kapitel 4.2: Datensätze aufteilen (Abschnitt: Die k-fache Kreuzvalidierung kurz erklärt)](../Kap04.2_Datens%C3%A4tze_aufteilen/%C3%9Cbung_Datens%C3%A4tze_aufteilen.ipynb).

Der folgende Code definiert eine **neue Funktion** `TrainKFold(...)`, die wir im Anschluss benutzen werden, um
* die Eingabe- und Ausgabedaten in *folds* aufzuteilen, mit diesen
* mehrfach das übergebene Modell zu trainieren,
* mit den tatsächlichen und vorhergesagten Ausgaben des Modells eine Konfusionsmatrix zu erstellen und
* daraus zum Schluss die resultierende Genauigkeit, Präzision und Sensitivität zu berechnen.

In [ ]:
import matplotlib.pyplot as plt
from   sklearn.model_selection    import KFold
from   sklearn.metrics            import confusion_matrix, ConfusionMatrixDisplay
from   sklearn.utils.class_weight import compute_sample_weight # die Methode verwenden wir erst zum Ende der Übung

# TrainKFold(k, model, X, y, class_weight=None)
#   Implementiert das Training des Modells 'model' mit einer 'k'-fachen Kreuzvalidierung
#   mit den Eingabewerten 'X' und Ausgabewerten 'y'
#   k     - Anzahl der folds für die Kreuzvalidierung
#   model - Klassifikations-Modell
#   X     - Eingabedaten
#   y     - Ausgabedaten
#   class_weight - "balanced" oder None
#                  Gewichtungs-Strategie bei unausgewogenen Ausgangsdaten
def TrainKFold(k, model, X, y, class_weight=None):
    # Mit KFold eine k-fache Kreuzvalidierung definieren
    folds = KFold(n_splits=k, shuffle=True, random_state=4)

    # Konfusionsmatrix 
    cm_all = [[0,0],[0,0]]
    # Über alle Split-Kombinationen der Eingaben (X) und Ausgaben (y) iterieren und jeweils:
    for (i_train, i_val) in folds.split(X):
        # per '.fit()'-Methode mit den Trainigsdaten trainieren. Achtung: i_train sind nur die Indizes, nicht die Daten
        if class_weight == None:
            model.fit(X.iloc[i_train], y.iloc[i_train])
        else:
            y_weights = compute_sample_weight(class_weight=class_weight, y=y.iloc[i_train])
            model.fit(X.iloc[i_train], y.iloc[i_train], y_weights)
        
        y_true = y.iloc[i_val].values
        y_pred = model.predict(X.iloc[i_val])
        
        cm = confusion_matrix(y_true, y_pred)
        # Confusion-Matrix: cm[actual,prediction]
        #  actual
        #    0    [ 0,0 ]=TN  [ 0,1 ]=FP
        #    1    [ 1,0 ]=FN  [ 1,1 ]=TP
        #            0           1      <== prediction
        
        accuracy  = (cm[1,1]+cm[0,0])/cm.sum()*100
        #precision= cm[1,1]/cm[:,1].sum()*100
        #recall   = cm[1,1]/cm[1,:].sum()*100
        print(f" - Modellgenauigkeit auf den fold-Validierungsdaten liegt bei {accuracy:03.2f}%")

        cm_all  += cm

    cm = cm_all
    ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['no error', 'defect']).plot(cmap=plt.cm.Blues)
    plt.show()
    accuracy  = (cm[1,1]+cm[0,0])/cm.sum()*100
    precision = cm[1,1]/cm[:,1].sum()*100
    recall    = cm[1,1]/cm[1,:].sum()*100
    print(f"Die mittlere Genauigkeit  aller Modelle liegt bei {accuracy:3.2f}%")
    print(f"Die mittlere Präzision    aller Modelle liegt bei {precision:3.2f}%")
    print(f"Die mittlere Sensitivität aller Modelle liegt bei {recall:3.2f}%")

*Hinweis:* Der obige Code erzeugt hier noch keine Ausgabe. Er definiert nur die neue Funktion.

Nachdem die Funktion nun definiert ist, können wir
* ein Modell - wieder einen Entscheidungsbaum - mit seinen Modellparametern erstellen, 
* über die neue Funktion *k=10* mal trainieren und validieren,
* die über alle Trainings akkumulierte Konfusionsmatrix anzeigen und
* Genauigkeit, Präzision und Sensitivität anzeigen lassen.

In [ ]:
model_dtree = tree.DecisionTreeClassifier(max_depth=3, random_state=16)
TrainKFold(10, model_dtree, X, y)

Wenn du nun die Maximale Tiefe des Entscheidungsbaums variierst, stellst du fest, dass die resultierende Genauigkeit sich nicht substanziell steigern lässt, jedoch Präzision und Sensitivität sich ändern.

Wir haben bislang noch nicht verglichen, wie sich eine *Reduktion der Merkmale* (die wir in `Xf` gespeichert haben) auswirkt. Dies sehen wir uns jetzt an und tauschen `X` durch `Xf`:

In [ ]:
model_dtree = tree.DecisionTreeClassifier(max_depth=3, random_state=16)
TrainKFold(10, model_dtree, Xf, y) # hier verwenden wir Xf (das im Vergleich zu X nur 7 Merkmale enthält)

Wir sehen, dass sowohl Präzision wie auch die Sensitivität leicht besser geworden sind.

## Aufgabe 5: Mehr Aufbereitung der Daten

Auch, wenn die Präzision etwas besser geworden ist, haben wir vielleicht die Vermutung, dass unsere Trainingsdaten noch ungewöhnliche oder extreme Einträge enthalten. 
Wir versuchen daher festzustellen, ob diese **Ausreißer** enthalten und **entfernen** diese.

Mit [describe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)() können wir uns zunächst einige statistische Werte berechnen lassen:

In [ ]:
df.describe()

Die Tabelle zeigt für jedes Merkmal die *Anzahl* (count), den *Mittelwert* (mean) und die *Standardabweichung* (std) der Werte, sowie den *minimalen* (min) und *maximalen* (max) vorkommenden Wert.

Die Zeilen *25%*, *50%* und *75%* grenzen die sogenannten *Quartilsbereiche* ab. Für das Merkmal **loc** (lines of code), das wir in der ersten Spalte sehen, bedeuten die Zahlen folgendes:
* (**count**) Es sind insgesamt `8927` Werte im Datensatz.
* (**min**) Der kleinste vorkommende Wert ist `0`.
* (**max**) Der größte vorkommende ist `3422`.
* (**25%**) ein Viertel aller Werte in der Spalte sind kleiner als `14.0` (also zwischen 0 und 14)
* (**50%**) die Hälfte aller Werte ist kleiner als `25.0` (die andere Hälfte ist größer). Dieser Wert wird auch als *Median* bezeichnet
* (**75%**) ein Viertel aller Werte ist größer als `50.0` (also zwischen 50 und 3442)

Der Bereich zwischen 25% und 75% wird *Interquartilsbereich* (IQR = Interquartile-Range) genannt und beinhaltet die Hälfte aller Werte des Merkmals. Sehen wir uns den mit dem **Bloxplot** eine grafische Darstellung dieses Bereiches an, den wir hier für das Merkmal **i** darstellen:

In [ ]:
plt.figure(figsize=[15,2], dpi=80)   # dies bereitet eine praktikable Darstellung der Grafik vor
print(df.describe()['i'])            # wir lassen uns hiermit die Statistik zum merkmal 'i' ausgeben
sns.boxplot(data=df, x='i');         # diese Zeile erzeugt den Boxplot - nur für das Merkmal 'i'

Wenn du oben die ausgegebenen statistischen Werte des Merkmals **i** ansiehst und grob mit dem Boxplot vergleichst, erkennst du, dass die blaue "Box" genau den *Interquartilsbereich* ($\text{IQR}=q_{75}-q_{25}$ = `43.675000 - 17.640000 = 26.035000`) belegt.

Die T-förmigen Begrenzungen rechts und links neben der Box werden durch "anhängen" eines Vielfachens (meist 1.5-fach) des $\text{IQR}$s. Diese Grenzen bestimmen dann, ob ein Wert als **Ausreißer** eingestuft wird oder nicht. Sie heißen daher manchmal auch *outlier fence*.

Die obere Grenze ist in diesem Fall bei: $q_{75} + 1.5\cdot\text{IQR}$ = `82.7275`.

In [ ]:
43.675000 + 1.5 * 26.035000

Wir  wollen diese Formel auf *alle Merkmale* anwenden. Dafür definieren wir zwei Funktionen, die die obere Grenze (*upper_fence*) und untere Grenze (*lower_fence*) anhand der Einträge unserer Statistik-Tabelle (siehe oben) berechnen. Dabei berücksichtigen wir, dass die obere Grenze nicht größer als der maximale Wert und die untere Grenze nicht kleiner als der minimale Wert des Merkmals werden soll.

In [ ]:
def upper_fence(q):
    iqr = q.loc['75%'] - q.loc['25%']
    return min( q.loc['75%'] + 1.5*iqr, q.loc['max'])

def lower_fence(q):
    iqr = q.loc['75%'] - q.loc['25%']
    return max( q.loc['25%'] - 1.5*iqr, q.loc['min'] )

Diese Funktionen wenden wir nun auf die Statistik-Tabelle `stat` an, um die numerischen Werte der beiden *Ausreißergrenzen* (*outlier fences*) für jedes Merkmal zu berechnen. Dafür nutzen wir die Methode [apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)() und lassen uns am Ende die oberen Grenzen ausgeben.

In [ ]:
stat = df.describe()
low  = stat.apply(lower_fence)
high = stat.apply(upper_fence)

high

Du erkennst beim Eintrag **i** wieder den gleichen, wie oben "von Hand" ausgerechneten Wert `82.7275`.

Bislang haben wir aber "nur" die Grenzen selbst berechnet. Nun müssen wir diese auf unseren Datensatz anwenden und für jeden einzelnen Eintrag und für jeden Merkmalswert prüfen ob dieser unter der unteren oder über der oberen *Ausreißergrenze* liegt.

Ist auch nur ein einziger Merkmalswert (also ein Wert einer Spalte) in einem Eintrag (also einer Zeile) außerhalb unserer beiden Grenzen, so wollen wir diesen als *Ausreißer* aus den Eingabewerten entfernen. Dazu nutzen wir die Methoden *less-than* [lt()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.lt.html) und *greater-than* [gt()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.gt.html), sowie wieder [any()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.any.html). value_counts() zeigt uns wieder an, wie viele der 8927 Einträge als *outlier* den Wert *True* bekommen haben.

In [ ]:
# Wir prüfen die Eingabewerte 'X', ob diese
# - kleiner als die untere Grenze '.lt(low)'
outlier = X.lt(low).any(axis=1) | X.gt(high).any(axis=1)
outlier.value_counts()

Hier sollten in der Ausgabe 3350 Einträge mit `True` gezählt worden sein.

Von den Eingabewerten, die wir in `X` gespeichert haben, selektieren wir nun die *Nicht*-Outlier und legen diese in `X2` ab. Ebenso müssen wir mit den Ausgabewerte `y` verfahren, deren verbleibende Ausgabewerte wir in `y2` ablegen.

In [ ]:
X2 = X[~outlier]
y2 = y[~outlier]

Wir prüfen gleich dein Ergebnis: Stimmt die Anzahl der, um die Ausreißer bereinigten Eingabe- und Ausgabewerte? Die Ausgabe sollte `(True, True)` ergeben:

In [ ]:
X2.shape == (5577,20), y2.shape == (5577,)

## Aufgabe 6: Modelle mit den zusätzlich aufbereiteten Daten

### 1. Entscheidungsbaum mit allen Merkmalen

Wir verwenden zunächst wieder einen Entscheidungsbaum, den wir - so wie zuletzt - mit der maximalen Tiefe 3 trainieren (10-fache Kreuzvalidierung). Nun verwenden wir aber die um die *Ausreißer* bereinigten Daten `X2` und `y2`.

In [ ]:
model_dtree = tree.DecisionTreeClassifier(max_depth=3, random_state=16)
TrainKFold(10, model_dtree, X2, y2)

Zum Vergleich lag die Präzision des Trainings mit den Ausreißern bei ca. 54%. Allerdings ist die Sensitivität dramatisch abgefallen: nur noch 1.56% aller fehlerhaften Softwaremodule würden jetzt als solche erkannt.

### 2. Entscheidungsbaum mit wenigen Merkmalen
Wie sieht das Ergebnis aus, wenn wir statt aller Merkmale wieder nur eine Auswahl treffen (die sieben in der `table` als Index gelisteten Merkmale)?

In [ ]:
X2f = X2.filter(items=table.index)
TrainKFold(10, model_dtree, X2f, y2)

Wenn du die letzten beiden Konfusionsmatrizen vergleichst, siehst du, dass *nur zwei* der 13 als fehlerhaft vorhergesagten Softwaremodule nun korrekt als nicht-fehlerhaft klassifiziert worden sind. Bei der sehr kleinen Anzahl an Fehlerprognosen macht dies jedoch eine um etwa 4% höhere Präzision aus!

### 2. Neuronales Netz
Wir wollen nun ein für Klassifikationen vorbereitetes "tiefes" neuronales Netz (NN, oder Multilayer Perceptron) trainieren. Wir erhoffen uns, dass durch die größere Anzahl an Modellparametern die Komplexität der Aufgabe vielleicht besser erfasst wird.

Wegen des Risikos eines Overfittings gehen wir aber auch hier bei der Evaluierung mit der 10-fachen Kreuzvalidierung vor. Wir können auch direkt die zuvor vorbereitete Funktion `TrainKFold` wiederverwenden.

Als Algorithmus verwenden wir den *MLPClassifier* der Scikit-Learn bibliothek: [Multilayer Perceptron Klassifikator](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier). Etwas [Hintergrundinformation](https://scikit-learn.org/stable/modules/neural_networks_supervised.html) zum "MLP" in der *scikit-learn* Bibliothek.

In [ ]:
from sklearn import neural_network as nn

model_nn = nn.MLPClassifier(hidden_layer_sizes=(10,8,5),  # <--- hier ist die Neuronenanzahl jeder versteckten Schicht aufgelistet
                            activation='relu', 
                            solver='adam',
                            validation_fraction=0.1,
                            alpha=0.0001, 
                            learning_rate='adaptive',     # (constant, invscaling, adaptive)
                            learning_rate_init=0.001,
                            max_iter=250,
                            shuffle=True,
                            random_state=4)
TrainKFold(10, model_nn, X2, y2)

Wir sehen hier, dass die *Präzision* nochmal besser geworden ist und sogar bei 65% liegt (die Sensitivität ist nach wie vor extrem klein). Diese scheinbare Güte in der Präzision kann aber auch täuschen, denn, wie beim Entscheidungsbaum zuvor, sind hier nur wenige Daten beteiligt und sprechen nicht für ein verlässliches Ergebnis.

Du kannst gerne an der Parametrisierung des NN spielen und so versuchen noch besser zu werden... oder auch schlechter.

Auch für das gleiche NN versuchen wir mit einem reduzierten Satz an Merkmalen `X2f` das Training:

In [ ]:
TrainKFold(10, model_nn, X2f, y2)

Wenn du die Modell-Hyperparameter nicht verändert hast, siehst du, dass - anders als beim Entscheidungsbaum zuvor - die Präzision schlechter wurde. Es ist also nicht immer so, dass eine einmal getroffene Auswahl von Merkmalen für *jeden* Algorithmus *gleichermaßen* eine Verbesserung bedeuten muss!

## Aufgabe 7: Unausgeglichene Daten angleichen (Entscheidungsbaum)
Wir haben einen Schritt der Datenvorbereitung bislang unterschlagen: Das Erheben einer repräsentativen Stichprobe, auch *sampling* genannt.

Beim Training eines Entscheidungsbaumes wirkt dies insbesondere auf die Wichtigkeit von Entscheidungsschwellen. Weil wir in unserem Datensatz vergleichsweise wenig Softwaremodule *mit* Fehlerzuständen haben, wird eine Erkennung von diesen - zugunsten einer besseren Genauigkeit - nicht so wichtig genommen.

Der Trainingsalgorithmus des `DecisionTreeClassifier` erlaubt es für die Einträge in den Trainingsdaten mit einem Korrekturfaktor zu gewichten - so als ob die häufig vorkommenden Klassen (in unserem Fall `false`) reduziert und die wenig vorkommenden (`true`) angereicht wären. In der von uns geschriebenen Funktion `TrainKFold` können wir dieses Feature mit dem zusätzlichen Argument `class_weight="balanced"` einschalten:

In [ ]:
model_dtree = tree.DecisionTreeClassifier(max_depth=3, random_state=16)
TrainKFold(10, model_dtree, X2, y2, class_weight="balanced")  # Modelltraining mit allen Merkmalen
TrainKFold(10, model_dtree, X2f, y2, class_weight="balanced") # Modelltraining mit den ausgewählten Merkmalen


Du siehst, dass hier signifikant mehr *Wahr-Positive* Klassifikation (543) stattfinden als zuvor und insbesondere die Sensitivität nun merklich gestiegen ist.